In [1]:
print('hola')

hola


In [81]:
from azureml.core import Workspace, Dataset

subscription_id = '20d4fdf3-6a4b-4f0b-a842-bd7392136332'
resource_group = 'cienciadatos'
workspace_name = 'azureml'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_id(workspace, '9227d1d4-d528-48db-84ed-768975adbf63')
dataset.download(target_path='.', overwrite=False)

['c:\\Users\\pablo.tamayo\\Desktop\\DataSchool\\Azure\\AzureDataSchool\\train_data.csv']

In [ ]:
##%%writefile experiments/mlflow_preprocess.py
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
import mlflow
import numpy as np
import pandas as pd

df = pd.read_csv("../airlines_delay/airlines_delay.csv", sep = ",")

def clean_data(data:pd.DataFrame) -> pd.DataFrame:
    '''
    Delete the Fligh column
    input: dataset
    output: pandas dataframe withiout Flight
    '''
    clean_df = data.drop('Flight', axis=1, inplace=False)
    return clean_df


def preprocess_data(data:pd.DataFrame) -> pd.DataFrame:
    '''
    Preprocess the data converting the categorical columns to one hot encodig
    and apply standarScaler to numeric columns
    input: pandas dataFrame
    output: tuple, first element corrempond to pd.Dataframe of independent features
            second element is the dependent(target) feature 
    '''

    numeric_columns = ['Length','Time']
    category_columns = ['Airline', 'AirportFrom', 'AirportTo', 'DayOfWeek']

    numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                            ('scaler', StandardScaler())])

    categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first'))])

    preprocessor = ColumnTransformer(
                        transformers=[
                            ('numeric', numeric_transformer, numeric_columns),
                            ('cat', categorical_transformer, category_columns)
                        ],
                        remainder='passthrough'
                    )
    
    #split the dependent(y) and independent(df_x) features 
    df_X = data.drop('Class', axis=1, inplace=False)
    #y_labels = data['Class']
    y = data['Class']
    preprocessed_data = preprocessor.fit_transform(df_X)
    encoded_category = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(category_columns)

    labels = np.concatenate([numeric_columns, encoded_category])
    
    preprocessed_df = pd.DataFrame(data=preprocessed_data, columns=labels)
    return pd.concat([preprocessed_df, y], axis=1)



df_new = clean_data(df)
processed_df= preprocess_data(df_new)
processed_df.head()

#df_new.head()

In [3]:
import mlflow

In [ ]:
#%%writefile experiments/mlflow_eda.py

from funct.myfuncs import cln_data
import mlflow
import argparse

experiment_name ='exp-Airlines'
mlflow.set_experiment(experiment_name)

with mlflow.start_run() as run:
    
    #loading data 
    #dataset = Dataset.get_by_name(ws, name='AirlinesDelay')
    df = pd.read_csv("../airlines_delay/airlines_delay.csv", sep = ",")
    
    #Distribution of the target column
    class_cero, class_one = df['Class'].value_counts()
    mlflow.log_metrics({'one':class_one, 'cero':class_cero})
    
    #Comparation of the mean
    mean_comparison = df[['Time','Length', 'Class']].groupby('Class').mean().to_dict()
 
    mlflow.log_metrics({'Mean Time class cero':mean_comparison['Time'][0], 'Mean Time class one':mean_comparison['Time'][1]})
    mlflow.log_metrics({'Mean Lenght class cero':mean_comparison['Length'][0], 'Mean Length class one':mean_comparison['Length'][1]})

    #Number of airlines
    number_of_airlines = len(df['Airline'].unique())

    mlflow.log_metric('number of airlines', number_of_airlines)

    #flights per airlnes
    flights_per_airline = df['Airline'].value_counts().to_dict()
    mlflow.log_metrics(flights_per_airline)

    
    #airports
    number_of_airports = len(df['AirportFrom'].unique())
    mlflow.log_metric('number of airports', number_of_airports)

    #airports with more activity
    arports_activity_df = df['AirportFrom'].value_counts().sort_values(ascending=False) + df['AirportTo'].value_counts().sort_values(ascending=False)
    top_5_airpots_activity = arports_activity_df.sort_values(ascending=False).head(5).to_dict()

    top_and_lowest_ariports_activity = {}
    for k, _ in top_5_airpots_activity.items():
        top_and_lowest_ariports_activity[f'Airport {k}'] = top_5_airpots_activity[k]


    #airports with more activity
    lowest_5_airpots_activity = arports_activity_df.sort_values(ascending=False).tail(5).to_dict()
    for k, _ in lowest_5_airpots_activity.items():
        top_and_lowest_ariports_activity[f'Airport {k}'] = lowest_5_airpots_activity[k]

    mlflow.log_metrics(top_and_lowest_ariports_activity)

    #DaysWeek
    dict_day_of_week= {1:'Monday', 2:'Tuesday', 3:'Wednesday', 4:'Thursday', 5:'Friday', 6:'Saturday', 7: 'Sunday'}
    df_day_of_week = df.replace({"DayOfWeek": dict_day_of_week},inplace=False)
    fligths_per_day_of_week = df_day_of_week['DayOfWeek'].value_counts().to_dict()

    mlflow.log_metrics(fligths_per_day_of_week)


    #plot distribution of time
    fig = plt.figure(figsize=(6,6))
    ax = fig.gca()    
    plt.hist(df['Length'])
    plt.show()
    mlflow.log_figure(fig, "plot.png")
    print("Plot listo!")

In [1]:
import pandas as pd

df = pd.read_csv('train_data.csv', delimiter=',')


In [ ]:
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_validate, RandomizedSearchCV
from azureml.core import Run
import pandas as pd
import numpy as np
import argparse
import os
import logging

x = df.drop('Class', axis=1, inplace=False)
y = df['Class']

'''
lr_cv = cross_validate(LogisticRegression(max_iter=400), x, y, cv=2, scoring=['balanced_accuracy', 'recall', 'precision'], return_train_score=True)
print('Listo regresion lineal')
#rf_cv = cross_validate(RandomForestClassifier(), x, y, cv=3, scoring=['balanced_accuracy', 'recall', 'precision'], return_train_score=True)
#print('Listo random forest')
svm = cross_validate(SVC(), x, y, cv = 3, scoring=['balanced_accuracy', 'recall', 'precision'], return_train_score=True)
print('Listo SVM')

hola =[]
for score_model, name_model in [(lr_cv, 'lr'), (svm, 'svm')]:
    hola.append({
        f'train_acc_{name_model}': np.mean(score_model['train_balanced_accuracy']),
        f'train_recall_{name_model}': np.mean(score_model['train_recall']),
        f'train_precision_{name_model}': np.mean(score_model['train_precision']),
        f'test_acc_{name_model}': np.mean(score_model['test_balanced_accuracy']),
        f'test_recall_{name_model}': np.mean(score_model['test_recall']),
        f'test_precision_{name_model}': np.mean(score_model['test_precision'])
    })


'''

param_dist ={
    'penalty':['l1', 'l2', 'elasticnet'],
    'C': [c/10 for c in range(1,20,5)],
    'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']

}
clf = RandomizedSearchCV(LogisticRegression(), param_dist, n_iter=5, random_state=0, refit='balanced_accuracy')
'''
mlflow.log_params({
    'penalty': 'l2',
    'C': 1.0,
    'solver':'lbfgs'
})
'''
clf.fit(x,y)

#mlflow.sklearn.log_model(lr, artifact_path='skleran-model')
y_pred = clf.predict(x)

recall = recall_score(y,y_pred)
precision = precision_score(y, y_pred)
acc = accuracy_score(y, y_pred)
conf_matrix = confusion_matrix(y, y_pred)
print(recall)
print(precision)
print(acc)
print(conf_matrix)

#logging.info(f'matriz de confusion{conf_matrix}')

'''
mlflow.log_metrics({
    'recall':recall,
    'precision': precision,
    'accuracy': acc,

})
'''

In [12]:
clf.best_params_

{'solver': 'newton-cg', 'penalty': 'l2', 'C': 0.6}

In [1]:
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_validate, RandomizedSearchCV
from azureml.core import Run
import pandas as pd
import numpy as np
import argparse
import os
import logging

x = df.drop('Class', axis=1, inplace=False)
y = df['Class']


svm = SVC()

svm.fit(x,y)

y_pred = svm.predict(x)

recall = recall_score(y,y_pred)
precision = precision_score(y, y_pred)
acc = accuracy_score(y, y_pred)
conf_matrix = confusion_matrix(y, y_pred)
print(recall)
print(precision)
print(acc)
print(conf_matrix)

ModuleNotFoundError: No module named 'sklearn'

In [4]:

from azureml.core import Workspace, Datastore, Run, Dataset
from sklearn.metrics import balanced_accuracy_score
import pandas as pd
from src.utils import dataset_transform
import argparse
import os
import mlflow

#run = Run.get_context()

ws = Workspace.from_config()
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
#raw_data_test = run.input_datasets['raw_test_data'].to_pandas_dataframe()


with mlflow.start_run():
    
    #load the latest version of the model
    model_name = 'airlines_model'
    model = mlflow.sklearn.load_model(f'models:/{model_name}/latest')
    print('loaded model')
    #mlflow.log_metric('prueba',1)
    dataset = Dataset.get_by_name(ws, name='airlines_train_data')
    df_all = dataset.to_pandas_dataframe()

#pred = model.predict(transformed_data_test.drop('Class', axis=1, inplace=False))

#acc = balanced_accuracy_score(transformed_data_test['Class'], pred)


loaded model


In [5]:
df_all.head()

,Time,Length,Airline,AirportFrom,AirportTo,DayOfWeek,Class
0,825.0,160.0,AA,DFW,DCA,1,1
1,1155.0,130.0,DL,ATL,PHL,4,0
2,965.0,70.0,WN,SJC,ONT,1,1
3,800.0,105.0,WN,SJC,PDX,5,1
4,525.0,105.0,AA,DFW,OMA,6,0


In [14]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import recall_score, precision_score, accuracy_score, confusion_matrix
import numpy as np
import joblib
import os
def dataset_transform_prueba(train_df=pd.DataFrame([]), val_df=pd.DataFrame([]), test_df=pd.DataFrame([])):

    numeric_columns = ['Length','Time']
    category_columns = ['Airline', 'AirportFrom', 'AirportTo', 'DayOfWeek']
    
    oh_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first')
    sc = StandardScaler()

    if (not(train_df.empty) and not(val_df.empty) and test_df.empty):
        
        #transforming train data
        X_train = train_df.drop('Class', axis=1, inplace=False)
        y_train = train_df['Class']

        X_train_oh = oh_encoder.fit_transform(X_train[category_columns])
        X_train_sc = sc.fit_transform(X_train[numeric_columns])
        one_hot_columns_name = oh_encoder.get_feature_names_out()

        train_oh_df = pd.DataFrame(data = X_train_oh, columns = one_hot_columns_name)
        train_sc_df = pd.DataFrame(data = X_train_sc, columns=numeric_columns)

        train_df = pd.concat([train_sc_df, train_oh_df, y_train], axis=1)

        #transformig val data
        X_val = val_df.drop('Class', axis=1, inplace=False)
        y_val = val_df['Class']

        X_val_oh = oh_encoder.transform(X_val[category_columns])
        X_val_sc = sc.transform(X_val[numeric_columns])
        one_hot_columns_name = oh_encoder.get_feature_names_out()

        val_oh_df = pd.DataFrame(data = X_val_oh, columns = one_hot_columns_name)
        val_sc_df = pd.DataFrame(data = X_val_sc, columns=numeric_columns)

        val_df = pd.concat([val_sc_df, val_oh_df, y_val], axis=1)


        #save the encoder and scaler
        joblib.dump(oh_encoder, 'oh_encoder.joblib')
        joblib.dump(sc, 'standarScaler.joblib')


        return train_df, val_df
    
    elif (train_df.empty and val_df.empty and not(test_df.empty)):
        test_df.reset_index(inplace=True)
        
        X_test= test_df.drop('Class', axis=1, inplace=False)
        y_test = test_df['Class']

        runs = mlflow.search_runs(experiment_names=["exp-Airlines"])
        recent_run = runs[['run_id', 'end_time' ]].loc[(runs['tags.mlflow.source.name'] == 'scripts/data_transform.py') & (runs['status'] == 'FINISHED')].sort_values(by='end_time', ascending=False).head(1)
        recent_run_id = recent_run['run_id'].to_string().split()[1]

        a = mlflow.artifacts.download_artifacts(run_id=recent_run_id, artifact_path='oh_encoder.joblib')
        b = mlflow.artifacts.download_artifacts(run_id=recent_run_id, artifact_path='standarScaler.joblib')

      
        sc_t = joblib.load(b)
        oh_t = joblib.load(a)

        sc_data_test = sc_t.transform(X_test[numeric_columns])
        oh_data_test = oh_t.transform(X_test[category_columns])
        one_hot_columns_name_test = oh_t.get_feature_names_out()

        test_oh_df = pd.DataFrame(data = oh_data_test, columns = one_hot_columns_name_test)
        test_sc_df = pd.DataFrame(data = sc_data_test, columns=numeric_columns)

        test_df_p = pd.concat([test_sc_df, test_oh_df, y_test], axis=1)

        return test_df_p
    
    else:
        return 'Worng combination'

In [165]:
b,c = dataset_transform_prueba(train_df=df_all, val_df=df_all)



In [166]:
b.head()

,Length,Time,Airline_DL,Airline_MQ,Airline_OO,Airline_WN,AirportFrom_ABI,AirportFrom_ABQ,AirportFrom_ACT,AirportFrom_ACV,...,AirportTo_VPS,AirportTo_XNA,AirportTo_YUM,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7,Class
0,0.468432,0.083172,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.018404,1.287022,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
2,-0.881652,0.593896,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,-0.356620,-0.008029,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
4,-0.356620,-1.011238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0


In [15]:
t = dataset_transform_prueba(test_df=df_all[40:41])

917980f1-9e6a-4eba-8da8-dca7161e96af
345    
Name: artifact_uri, dtype: object


In [16]:
t

,Length,Time,Airline_DL,Airline_MQ,Airline_OO,Airline_WN,AirportFrom_ABI,AirportFrom_ABQ,AirportFrom_ACT,AirportFrom_ACV,...,AirportTo_VPS,AirportTo_XNA,AirportTo_YUM,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7,Class
0,-0.881652,0.028451,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0


In [7]:
runs = mlflow.search_runs(
        experiment_names=["exp-Airlines"]
    )
runs.to_csv('runs.csv')

In [8]:
recent_run = runs[['run_id', 'end_time' ]].loc[(runs['tags.mlflow.source.name'] == 'scripts/data_transform.py') & (runs['status'] == 'FINISHED')].sort_values(by='end_time', ascending=False).head(1)
recent_run_id = recent_run['run_id'].to_string().split()[1]

In [9]:
recent_run_id

'917980f1-9e6a-4eba-8da8-dca7161e96af'

In [219]:
hola = Run.get_file_names()

TypeError: get_file_names() missing 1 required positional argument: 'self'

'C:\\Users\\PABLO~1.TAM\\AppData\\Local\\Temp\\tmpsgg_o2hr\\oh_encoder.joblib'